In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *

In [3]:
# df = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')

# df = pd.DataFrame(df.groupby(['article'])['related'].apply(list).reset_index()['related'])

# df['num_of_links'] = df['related'].apply(len)

# def count_True(l):
#     total = len(l)
#     trues = len([1 for e in l if e])
#     return trues/total


# df['perc'] = df['related'].apply(count_True)

# df.to_csv('/Users/smeros/Desktop/test.tsv', sep='\t', index=None)

# df[df['num_of_links']==4].to_csv('/Users/smeros/Desktop/test_4.tsv', sep='\t', index=None)